# Supplemental exercise for root-finding via Newton-Raphson

*Last updated by Christian Cahig on 2024-10-21.*

## Preliminaries

SciPy provides root-finding tools within its `optimize` module
(hence, [`scipy.optimize`](https://docs.scipy.org/doc/scipy/reference/optimize.html)).
Specifically, we will use the function
[`scipy.optimize.newton`](https://docs.scipy.org/doc/scipy/reference/generated/scipy.optimize.newton.html).

In [1]:
import math as mt
import numpy as np
import scipy.optimize as spopt

## Scenario

We will use as example Part 2 of [Laboratory Activity 1](./act-01/act-01.ipynb)
(see also the [model answers](./act-01/act-01_ans.ipynb)),
*i.e.*, solving for $\theta_{1}$ in 
$$
O\cos\theta_{1} + P\sin\theta_{1} + Q = 0,
$$

where:

$$
L = F_{3} \cos\theta_{3} - \frac{r_{w}}{\sqrt{r_{w}^{2} + r_{s}^{2}}} F_{4},
\\
M = F_{3} \sin\theta_{3} - \frac{r_{s}}{\sqrt{r_{w}^{2} + r_{s}^{2}}} F_{4},
\\
O = \frac{F_{1}}{\sin\theta_{2}},
\quad
P = \frac{F_{1}}{\cos\theta_{2}},
\quad
Q = \frac{L}{\sin\theta_{2}} - \frac{M}{\cos\theta_{2}},
$$

for:
$v = 17.09$,
$F_{1} = 4$,
$F_{3} = 5$,
$F_{4} = 7$,
$\theta_{2} = 70$,
$\theta_{3} = 30$,
$r_{s} = 3$, and
$r_{w} = 4$.
Hence, the problem is formulated as finding the root of $g\!\left(\theta_{1}\right) = O \cos\theta_{1} + P \sin\theta_{1} + Q$.

## Implementation

In the [model answers](./act-01/act-01_ans.ipynb),
we implemented $g\!\left(\theta_{1}\right)$
as a Python function `residual_from_theta1`
accepting a single argument `theta1`,
with the values of $L$, $M$, $O$, $P$, and $Q$
computed prior to the definition of `residual_from_theta1`.
This is fine for beginners,
but we don't want to remain beginners.

Now, we will implement `residual_from_theta1` such that it
accepts a positional argument `theta1`
representing the variable $\theta_{1}$,
and some keyword arguments representing
the variables with given values.

In [2]:
def residual_from_theta1(
    theta1,
    F1 = 4.0,
    F3 = 5.0,
    F4 = 7.0,
    theta2 = mt.radians(70.0),
    theta3 = mt.radians(30.0),
    rs = 3.0,
    rw = 4.0,
    use_radians = True,
):
    if not use_radians:
        # If the provided angles are not in radians, convert them to radians first.
        theta1 = mt.radians(theta1)
        theta2 = mt.radians(theta2)
        theta3 = mt.radians(theta3)

    L = (F3 * mt.cos(theta3)) - ((rw * F4) / mt.sqrt((rs**2) + (rw**2)))
    M = (F3 * mt.sin(theta3)) - (((rs * F4)) / mt.sqrt((rs**2) + (rw**2)))
    O = F1 / mt.sin(theta2)
    P = F1 / mt.cos(theta2)
    Q = (L / mt.sin(theta2)) - (M / mt.cos(theta2))
    
    return (O * mt.cos(theta1)) + (P * mt.sin(theta1)) + Q

With the above implementation,
computing $g\!\left(\theta_{1}\right)$ for various values of the "given" variables
can be invoked by, *e.g.*,

```python
residual_from_theta1(0.5, F1 = 3)
residual_from_theta1(30, rs = 2.0, rw = 3.5, use_radians = False)
```

Then, we need a Python function implementing the computation of the derivative

$$
\frac{dg\!\left(\theta_{1}\right)}{d\theta_{1}}
=
-O \sin\theta_{1} + P \cos\theta_{1}
$$

In [3]:
def deriv_residual_from_theta1(
    theta1,
    F1 = 4.0,
    theta2 = mt.radians(70.0),
    theta3 = mt.radians(30.0),
    use_radians = True,
):
    if not use_radians:
        # If the provided angles are not in radians, convert them to radians first.
        theta1 = mt.radians(theta1)
        theta2 = mt.radians(theta2)
        theta3 = mt.radians(theta3)

    O = F1 / mt.sin(theta2)
    P = F1 / mt.cos(theta2)

    return (-O * mt.sin(theta1)) + (P * mt.cos(theta1))

In [4]:
def F2_from_theta1(
    theta1,
    F1 = 4.0,
    F3 = 5.0,
    F4 = 7.0,
    theta2 = mt.radians(70.0),
    theta3 = mt.radians(30.0),
    rs = 3.0,
    rw = 4.0,
    use_radians = True,
    use_E_force_balance = True,
):
    if not use_radians:
        # If the provided angles are not in radians, convert them to radians first.
        theta1 = mt.radians(theta1)
        theta2 = mt.radians(theta2)
        theta3 = mt.radians(theta3)

    L = (F3 * mt.cos(theta3)) - ((rw * F4) / mt.sqrt((rs**2) + (rw**2)))
    M = (F3 * mt.sin(theta3)) - (((rs * F4)) / mt.sqrt((rs**2) + (rw**2)))
    O = F1 / mt.sin(theta2)
    P = F1 / mt.cos(theta2)

    if use_E_force_balance:
        return -(O * mt.cos(theta1)) - (L / mt.sin(theta2))
    return (P * mt.sin(theta1)) - (M / mt.cos(theta2))

Now let us run a Newton-Raphson
starting at an initial guess of 0,
and using the default tolerance and an iteration budget of 500.

*See
[the documentation for `scipy.optimize.newton`](https://docs.scipy.org/doc/scipy/reference/generated/scipy.optimize.newton.html)
for more details.*

In [5]:
Z_start = mt.pi
print(f"Finding a root starting at {Z_start} ...")
Z, Z_info = spopt.newton(
    residual_from_theta1,
    Z_start,
    fprime = deriv_residual_from_theta1,
    full_output = True, disp = False,
)

print(Z_info)
print(f"Residual value: {residual_from_theta1(Z)}")
print(
    "If the first cable pulls along the direction of",
    f"{Z} rad. ({mt.degrees(Z)}°) S of E,",
    "the second cable pulls with:\n"
    f"\ta {F2_from_theta1(Z, use_E_force_balance=True)}-kN force",
    "(according to the balance of eastward force components)\n"
    f"\ta {F2_from_theta1(Z, use_E_force_balance=False)}-kN force",
    "(according to the balance of northward force components)"
)

Finding a root starting at 3.141592653589793 ...
      converged: True
           flag: converged
 function_calls: 8
     iterations: 4
           root: 3.0875779938611365
         method: newton
Residual value: 0.0
If the first cable pulls along the direction of 3.0875779938611365 rad. (176.9051879657127°) S of E, the second cable pulls with:
	a 5.601873544906228-kN force (according to the balance of eastward force components)
	a 5.601873544906228-kN force (according to the balance of northward force components)


## Now it's your turn

Using `for` loop
and
the [`numpy.linspace`](https://numpy.org/doc/stable/reference/generated/numpy.linspace.html) function,
try different starting points uniformly sampled from the interval $\left[0, 2\pi\right]$ radians.
You may consult [this Real Python tutorial](https://realpython.com/np-linspace-numpy/)
as reference.

In [ ]:
for Z_start in np.linspace(0, 2*mt.pi, num = 10):
    print(f"Finding a root starting at {Z_start} ...")
    Z, Z_info = spopt.newton(
        residual_from_theta1,
        Z_start,
        fprime = deriv_residual_from_theta1,
        full_output = True, disp = False,
    )

    print(
        "If the first cable pulls along the direction of",
        f"{Z} rad. ({mt.degrees(Z)}°) S of E,",
        "the second cable pulls with:\n"
        f"\ta {F2_from_theta1(Z, use_E_force_balance=True)}-kN force",
        "(according to the balance of eastward force components)\n"
        f"\ta {F2_from_theta1(Z, use_E_force_balance=False)}-kN force",
        "(according to the balance of northward force components)"
    )